In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('data/high_diamond_ranked_10min.csv')

In [ ]:
diffcols = [c for c in df.columns if 'diff' in c.lower()]
diffcols

In [ ]:
onlyBlueCols = [c for c in df.columns if 'blue' in c.lower()]
onlyBlueCols

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(df[onlyBlueCols].corr(), annot=True, fmt=".2f")
plt.show()

In [ ]:
sns.jointplot(data=df, x="blueGoldDiff", y="blueExperienceDiff", hue="blueWins")
plt.show()

In [ ]:
sns.jointplot(data=df, x="blueDeaths", y="blueKills", hue="blueWins")
plt.show()

In [ ]:
features = df[['blueWins','blueGoldDiff', 'blueExperienceDiff']]

features["KDDiff"] = df["blueKills"] - df["blueDeaths"]
features["ADiff"] = df["blueAssists"] - df["redAssists"]
features["EMDiff"] = df["blueEliteMonsters"] - df["redEliteMonsters"]
features["TowersDiff"] = df["blueTowersDestroyed"] - df["redTowersDestroyed"]
features["MinionsDiff"] = df["blueTotalMinionsKilled"] - df["redTotalMinionsKilled"]

plt.figure(figsize=(12, 12))
sns.heatmap(features.corr(), annot=True, fmt=".2f")
plt.show()

### Description

League of Legends is a MOBA (multiplayer online battle arena) where 2 teams (blue and red) face off.

The game is about

- killing:

    - enemy players

    - minions

    - monsters in the jungle (some of them are elite: Dragons, Heralds)
    
- destroying buildings of enemy team.


Every kill or structure destruction gives both gold for a player or entire team, and experience for a player such that they can gain bigger level.
Every player gets one kill (K) while finishing enemy, one death (D) while dying (most often from an enemy) and assist (A) while helping teammate with kill.

The ultimate aim of the game is to destroy enemy main building.


## Model 1: Probit model



In [ ]:
from cmdstanpy import CmdStanModel

probit_model = CmdStanModel(stan_file='models/probit.stan')

data = {
    'N': len(features),
    'y': features['blueWin'].values.astype(int),
    'X': features.drop('blueWin', axis=1).values,
}

fit = probit_model.sample(data=data, iter_sampling=1000, chains=4)

print(fit.summary())


# Model 2: Logit model

In [ ]:
logit_model = CmdStanModel(stan_file='models/logit.stan')

data = {
    'N': len(features),
    'y': features['blueWin'].values.astype(int),
    'X': features.drop('blueWin', axis=1).values,
}

fit = logit_model.sample(data=data, iter_sampling=1000, chains=4)

print(fit.summary())